# How to use HSDatalog SDK module - \[OFFLINE Features\]
---
<br>
<p>Import the HSDatalog module</p>

In [ ]:
from HSD import HSDatalog as HSD

Initialize the HSDatalog by passing a valid acquisition folder as a parameter

In [ ]:
acq_folder = "..\\STWIN_acquisition_examples\\20200708_10_31_26"
acq_folder = "..\\STWIN_acquisition_examples\\20201110_16_22_19"
hsd = HSD.HSDatalog(acquisition_folder=acq_folder)

HSDatalog allows you to obtain all the information regarding the acquisition and the board that generated it.
- You can obtain the list of the available active sensors:

In [ ]:
active_sensor_list = hsd.getSensorList(only_active=True)

for s in active_sensor_list:
    print(s.name, end=', ')

- You can directly access the sensor passing its name to the following function

In [ ]:
sensor_name = "HTS221"
sensor = hsd.getSensor(sensor_name)


For each sensor it is possible to extract all the information regarding current configuration \[Status\] and capabilities \[Descriptor\]

In [ ]:
print ("Sensor: {}".format(sensor.name))

s_descriptor_list = sensor.sensor_descriptor.sub_sensor_descriptor
s_status_list = sensor.sensor_status.sub_sensor_status

for i, s in enumerate(s_descriptor_list):
    print(" --> {} - ODR: {} , FS: {} , SamplesPerTs {}".format(s.sensor_type,s_status_list[i].odr,s_status_list[i].fs,s_status_list[i].samples_per_ts))

You can also get the list of sensor data files in your selected acquisition folder:

In [ ]:
file_names = hsd.getDataFileList()
print(file_names)

Once you have collected all sensors desired parameters, you can extract data from your acquisition in several different ways:
# Data Extraction
---
## Pandas DataFrame
Data are extracted from the acquisition folder as pandas DataFrames. Select a sensor and get a DataFrame as shown below

In [ ]:
sensor_name = "IIS3DWB"
sensor_type = "ACC"
df = hsd.get_dataFrame(sensor_name, sensor_type)
df.describe()

If needed you can select a subset of the file. It can be useful to limit the amount of data to be analyzed in case of large files. 

In [ ]:
df = hsd.get_dataFrame(sensor_name,sensor_type, sample_start = 10, sample_end = 20)
df.describe()

Add labels to the exported DataFrame (if they are present in the acquisition)

In [ ]:
df = hsd.get_dataFrame(sensor_name, sensor_type, labeled = True)
print(df)

Raw data selection. Data will not be multiplied by the sensor sensitivity value

In [ ]:
df = hsd.get_dataFrame(sensor_name, sensor_type, raw_flag = True)
df.describe()

# Data Conversion
---
Starting from a DataFrame it is possible to convert the data of an acquisition in the following formats:
<ul>
<li> CSV, TSV
<li> Cartesiam format
<li> ST UNICO format (Work in progress)
</ul>
To access the conversion features, we have to:
<ul>
<li> Import the HSD_utils.converters module.
<li> choose a folder to save the conversion output files
</ul>


In [ ]:
from HSD_utils.converters import HSDatalogConverter
import os

output_folder = "..\\STWIN_acquisition_examples\\20200708_10_31_26\\exported_data_folder"
output_folder = "..\\STWIN_acquisition_examples\\20201110_16_22_19\\exported_data_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

Select a sensor and obtain a DataFrame from the corresponding acquisition file

In [ ]:
sensor_name = "IIS3DWB"
sensor_type = "ACC"
df = hsd.get_dataFrame(sensor_name, sensor_type)

## CSV, TSV Conversion

In [ ]:
exported_sensor_path = os.path.join(output_folder,sensor_name)

HSDatalogConverter.to_csv(df, exported_sensor_path)
HSDatalogConverter.to_tsv(df, exported_sensor_path)

## Cartesiam format Conversion

In [ ]:
HSDatalogConverter.to_cartesiam_format(output_folder, sensor_name, df, 150)

## ST UNICO format Conversion (Work in progress)

In [ ]:
hsd.to_unico(output_folder,sensor_name='ISM330DHCX')

# Plot
---
Using the HSDatalog module it is possible to visualize the data of an acquisition as follows
- You can plot one sensor:

In [ ]:
import matplotlib.pyplot as plt
sensor_name = "IIS3DWB"
sensor_type = "ACC"

hsd.get_sensorPlot(sensor_name, sensor_type)

Or all of them:

In [ ]:
active_sensor_list = hsd.getSensorList(only_active=True)

plots = []
for s in active_sensor_list:
    for ssd in s.sensor_descriptor.sub_sensor_descriptor:
        hsd.get_sensorPlot(s.name, ssd.sensor_type)

The module provides a set of options to personalize the plots
- It is possible to limit the plot to a specific samples interval

In [ ]:
sensor_name = "IIS3DWB"
sensor_type = "ACC"

hsd.get_sensorPlot(sensor_name, sensor_type, sample_start=50000, sample_end=100000)

- To display data annotations (acquisition must be annotated)

In [ ]:
label_list = hsd.getAcquisitionLabelClasses()
print(label_list)

sensor_name = "HTS221"
sensor_type = "TEMP"
hsd.get_sensorPlot(sensor_name, sensor_type, label='HW_TAG_0')

- To display a sub-plot for each dimension, in case of multidimensional sensors

In [ ]:
sensor_name = "ISM330DHCX"
sensor_type = "GYRO"
hsd.get_sensorPlot(sensor_name, sensor_type, subplots=True)

- To show raw data (not multiplied by sensitivity)

In [ ]:
sensor_name = "IIS3DWB"
sensor_type = "ACC"

sensor = hsd.getSensor(sensor_name)
sensor_sensitivity = sensor.sensor_status.sub_sensor_status[0].sensitivity
print("sensitivity: {}".format(sensor_sensitivity))

hsd.get_sensorPlot(sensor.name, sensor_type, raw_flag=True)